## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비
https://monetd.github.io/python/nlp/Word-Embedding-Word2Vec-%EC%8B%A4%EC%8A%B5/

https://comlini8-8.tistory.com/6

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/Othercomputers/내 PC/DEV/DSL/세션2/정규세션/[0830] TextMining_WordEmbedding/TextMining_WordEmbedding

/content/drive/Othercomputers/내 PC/DEV/DSL/세션2/정규세션/[0830] TextMining_WordEmbedding/TextMining_WordEmbedding


In [4]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [5]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [6]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [7]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])

corpus = sum(corpus, [])
len(corpus)

  0%|          | 0/50 [00:00<?, ?it/s]

1020

In [ ]:
test_list = [[1,2],[3,4,5]]
sum(test_list, []) # python list 합치는 방법

[1, 2, 3, 4, 5]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [8]:
corpus[:50]

['AI로 한국인·중국인·일본인들의 얼굴을 분석한 결과, 각 인종별로 몇가지 패턴이 존재했다',
 '(맨 윗줄부터)중국인, 일본인, 한국인./미국 로체스터대 컴퓨터학과 연구팀 논문',
 '얼굴 생김이 비슷한 한국인과 중국인, 일본인',
 '인공지능(AI)이 사람보다 더 정확하게 한·중·일 3국 인종을 구분한다는 연구 결과가 나왔다',
 '미국 워싱턴포스트는 “미국 로체스터대 컴퓨터학과 연구팀이 한국인과 중국인, 일본인의 얼굴 자료 4만여 건을 AI를 통해 구분한 결과 약 75％의 정확도로 국적을 구분했다”고 21일(현지시각) 보도했다',
 '75%의 정확도는 사람이 사진을 보고 한·중·일 국적을 구분해내는 확률(약 39％)의 두 배 수준이다',
 '연구팀은 리카이푸(李開復) 전 구글 차이나 사장의 트위터를 팔로우하는 중국인 가운데 1만3429명, 아베 신조(安倍晋三) 일본 총리 트위터를 팔로우하는 일본인 1만2914명, 박근혜 대통령 트위터를 팔로우하는 한국인 1만3540명 등 4만여명의 프로필 사진을 활용했다',
 '연구팀은 이들의 얼굴을 눈썹, 광대뼈, 머리카락색, 안경, 미소 등 40개 요소로 분류해 차이를 분석했다',
 '연구팀은 “얼굴에서는 헤어스타일이나 안경, 표정 등에서 차이가 뚜렷했고, 특히 한·중·일 3국은 패션감각에서 확연하게 다르다”고 밝혔다',
 '일본인은 눈 밑에 늘어진 지방층을 가진 경우가 가장 많았다',
 '중국인은 숱이 많은 눈썹을 가진 경우가 많았다',
 '웃는 얼굴은 남녀불문 일본인이 가장 많았고 중국인이 가장 적었다',
 '앞머리는 일본인 얼굴에서 가장 자주 보이고 중국인들 사이에서는 드물었다',
 '한국인은 검은 머리카락색이 많고, 일본에서는 상대적으로 검은 머리카락색이 적었다',
 '연구진은 “앞으로 이 기술을 활용하면 광고나 소셜미디어 마케팅에서 맞춤형 광고를 할 수 있다”면서 “테러리즘 예방에도 사용할 수 있다”고 했다',
 '파동에너지 마음대로 제어하는 신물질 개발',
 '박남규 서울대 교수팀 개발…광학·반도체 등에 

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]
* 가장 먼저 corpus에 정규표현식을 활용해 한글 문자, 공백, 숫자를 제외한 구두점, 특수문자 등을 제거함
* 그 다음 customtokenizer로 원치 않는 품사를 제외하여 tokenizing 수행

In [9]:
okt = Okt()

In [10]:
corpus[6]

'연구팀은 리카이푸(李開復) 전 구글 차이나 사장의 트위터를 팔로우하는 중국인 가운데 1만3429명, 아베 신조(安倍晋三) 일본 총리 트위터를 팔로우하는 일본인 1만2914명, 박근혜 대통령 트위터를 팔로우하는 한국인 1만3540명 등 4만여명의 프로필 사진을 활용했다'

In [11]:
# test - 문장을 okt를 통해 토큰화 해보자
print(okt.pos(corpus[6], stem  = True, norm = True))

[('연', 'Modifier'), ('구', 'Modifier'), ('팀', 'Noun'), ('은', 'Josa'), ('리카이푸', 'Noun'), ('(', 'Punctuation'), ('李開復', 'Foreign'), (')', 'Punctuation'), ('전', 'Noun'), ('구글', 'Noun'), ('차이나', 'Noun'), ('사장', 'Noun'), ('의', 'Josa'), ('트위터', 'Noun'), ('를', 'Josa'), ('팔로우', 'Noun'), ('하다', 'Verb'), ('중국인', 'Noun'), ('가운데', 'Noun'), ('1만', 'Number'), ('3429', 'Number'), ('명', 'Noun'), (',', 'Punctuation'), ('아베', 'Noun'), ('신조', 'Noun'), ('(', 'Punctuation'), ('安倍晋三', 'Foreign'), (')', 'Punctuation'), ('일본', 'Noun'), ('총리', 'Noun'), ('트위터', 'Noun'), ('를', 'Josa'), ('팔로우', 'Noun'), ('하다', 'Verb'), ('일본인', 'Noun'), ('1만', 'Number'), ('2914', 'Number'), ('명', 'Noun'), (',', 'Punctuation'), ('박근혜', 'Noun'), ('대통령', 'Noun'), ('트위터', 'Noun'), ('를', 'Josa'), ('팔로우', 'Noun'), ('하다', 'Verb'), ('한국인', 'Noun'), ('1만', 'Number'), ('3540', 'Number'), ('명', 'Noun'), ('등', 'Noun'), ('4만', 'Number'), ('여명의', 'Foreign'), ('프로필', 'Noun'), ('사진', 'Noun'), ('을', 'Josa'), ('활용', 'Noun'), ('하다', 'Verb')]


## 정규표현식
한글문자 제외한 구두점, 특수문자 제거

In [12]:
import re

In [13]:
re.sub("[^1-9ㄱ-ㅎㅏ-ㅣ가-힣 ]","",corpus[6])

'연구팀은 리카이푸 전 구글 차이나 사장의 트위터를 팔로우하는 중국인 가운데 1만3429명 아베 신조 일본 총리 트위터를 팔로우하는 일본인 1만2914명 박근혜 대통령 트위터를 팔로우하는 한국인 1만354명 등 4만여명의 프로필 사진을 활용했다'

In [14]:
corpus_reg = []
for i in tqdm(corpus):
  corpus_reg.append(re.sub("[^1-9ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ",i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [15]:
len(corpus_reg)

1020

## stopwords

In [16]:
stoppoom = ['Josa','Suffix','Punctuation', 'Foreign', 'Number']
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다'] # 원하지 않는 단어 제거도 추가

In [17]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence, stop1, stop2):
    tokenized = []
    for i,j in okt.pos(sentence, stem=True, norm=True):
        if j in stop1:
            continue
        if i in stop2: # 원하지 않는 단어 제거도 추가
            continue
        tokenized.append(i)
    return tokenized

In [18]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus_reg):
    corpus_tokenized.append(CustomTokenization(i, stoppoom, stopwords))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [19]:
corpus_tokenized[:2]

[['로',
  '한국인',
  '중국인',
  '일본인',
  '얼굴',
  '분석',
  '결과',
  '각',
  '인종',
  '별로',
  '몇',
  '가지',
  '패턴',
  '존재'],
 ['맨', '윗줄', '중국인', '일본인', '한국인', '미국', '로체스터', '컴퓨터', '학과', '연구', '팀', '논문']]

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

* window=3 : sliding window의 사이즈를 3으로 두어 단어 앞과 뒤에서 총 3개의 단어를 보고 판단할 것을 명시
* min_count=1 : 1번 이상 나타나면 vocab list에 추가 (1로 안두면 fine-tuning에서 에러남을 확인)
* sg=1 (default) : skipgram 모델 사용

## No Pre-trained

In [21]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size = 200, window=3, min_count=1) # size 200으로 설정

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 3597


In [22]:
model_no_pretrained.wv.vocab.keys()

dict_keys(['로', '한국인', '중국인', '일본인', '얼굴', '분석', '결과', '각', '인종', '별로', '몇', '가지', '패턴', '존재', '맨', '윗줄', '미국', '로체스터', '컴퓨터', '학과', '연구', '팀', '논문', '생기다', '비슷하다', '인공', '지능', '사람', '더', '정확하다', '중', '일', '국', '구분', '한다는', '나오다', '워싱턴포스트', '연', '구', '자료', '건', '통해', '약', '정확도', '국적', '고', '현지', '시각', '보도', '사진', '보고', '해내다', '확률', '두', '배', '수준', '리카이푸', '전', '구글', '차이나', '사장', '트위터', '팔로우', '가운데', '명', '아베', '신조', '일본', '총리', '박근혜', '대통령', '등', '프로필', '활용', '눈썹', '광대뼈', '머리카락', '색', '안경', '미소', '개', '요소', '분류', '차이', '헤어스타일', '표정', '뚜렷하다', '특히', '패션', '감각', '확연', '다르다', '밝히다', '눈', '밑', '늘어지다', '지방', '층', '가지다', '경우', '가장', '많다', '숱', '웃다', '남녀', '불문', '적다', '앞머리', '자주', '보이', '중국', '사이', '드물다', '검다', '상대', '구진', '앞', '기술', '광고', '소셜미디어', '마케팅', '맞춤', '수', '있다', '면서', '테러리즘', '예방', '사용', '파동', '에너지', '마음대로', '제어', '신', '물질', '개발', '박남규', '서울대', '교수', '광학', '반도체', '적용', '가능', '잔잔하다', '물', '돌', '던지다', '주위', '물결', '이다', '돌이', '일으키다', '물이', '전달', '때문', '말', '소리', '들다', '것', '공기', '이렇게', 

In [23]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 20)

(282749, 303740)

In [25]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('과실') # 딱히...

[('나서다', 0.9949696660041809),
 ('현지', 0.9949036240577698),
 ('사이언스', 0.9948916435241699),
 ('기존', 0.9948892593383789),
 ('그러나', 0.9948755502700806),
 ('노벨', 0.9948692321777344),
 ('관련', 0.9948513507843018),
 ('골', 0.9948458671569824),
 ('어렵다', 0.9948390126228333),
 ('얼굴', 0.9948297739028931)]

## Pre-trained

In [26]:
# 사전학습 된 모델을 사용하여 학습
# 사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', pretrained.wv.vector_size)

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200


In [27]:
# 모델 생성
model_pretrained = Word2Vec(size = pretrained.wv.vector_size, window=3, min_count=1)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

현재 vocabulary size(V) : 3597


In [28]:
len(pretrained.wv.vocab.keys())

30185

In [29]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

현재 vocabulary size(V) : 31224


In [30]:
#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

In [31]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 20)

(299746, 303740)

In [32]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('과실') # good result

[('업무상', 0.6928348541259766),
 ('손해', 0.6507189273834229),
 ('보증인', 0.6148687601089478),
 ('보험자', 0.6061666011810303),
 ('배임죄', 0.6038691997528076),
 ('피고인', 0.5997948050498962),
 ('수익자', 0.5995974540710449),
 ('기망', 0.5991066694259644),
 ('불이행', 0.5902891755104065),
 ('상해죄', 0.5902408361434937)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]

epoch 횟수 등 각종 hyperparameter는 둘다 동일하게 두고 학습을 진행하였다. 단어 '과실'을 기준으로 유사한 단어를 찾는 식으로 비교를 하였는데, 사전 학습 없이 모델을 학습한 경우에는 전혀 유사한 단어를 보여주지 못했다. 반면 사전 학습된 모델은 '손해', '불이행' 등 유사한 단어를 보여주었다. 이와 같이 성능 차이가 난 이유는 이미 학습된 모델이 적절한 vector값을 잘 가지고 있기에 가능한 것으로 생각된다.

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]
* t-SNE의 perplexity를 10으로 두고 군집화를 진행했는데, PCA는 전반적인 단어들이 한쪽으로 좀 몰린 반면, t-SNE는 고르게 퍼진 결과를 보였다.
* 유의미한 시각화 결과로는 PCA에서 윗부분에 '출범', '개최', '설립' 등의 단어들이 모여있으며, t-SNE에서 왼쪽 위에 '독일', '프랑스' ,'영국' 등의 단어들이 모여있다.

In [33]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, []))) # set을 통해 겹치는 단에 제외
len(unique_text)

3597

In [34]:
unique_text[:3]

['급박하다', '뇌혈관', '엠블럼']

In [35]:
model_pretrained.wv['급박하다']

array([ 0.01502851,  0.00751311, -0.01329149,  0.00854874, -0.02630479,
        0.01405895, -0.04726035,  0.05117818, -0.0060154 ,  0.00716657,
        0.01855579,  0.00668429, -0.02010135, -0.04749933, -0.02074013,
        0.01062282, -0.0164032 ,  0.01107146, -0.02094422,  0.00677177,
        0.00412014, -0.01568399,  0.03481089,  0.01156035,  0.01662469,
       -0.01733209,  0.00614086, -0.01690569, -0.02060783, -0.01410708,
       -0.00988281,  0.01027432, -0.00070834, -0.02523587,  0.00617421,
       -0.01313779, -0.00283741, -0.02593539,  0.02065738,  0.01232238,
       -0.01107045, -0.0483381 ,  0.04154188, -0.01446405, -0.00818637,
        0.01870313,  0.00088776, -0.00620531,  0.03144169, -0.01554214,
        0.00329849,  0.00756157,  0.01097938, -0.00913993,  0.00198278,
       -0.00982015,  0.03269846, -0.01918063, -0.00839442, -0.00900914,
       -0.01976465,  0.01918917, -0.01487471,  0.02609577, -0.03692761,
        0.02489098,  0.00873697,  0.04800472,  0.00609376, -0.00

In [36]:
model_pretrained.wv['급박하다'].shape # 200차원의 벡터로 구성

(200,)

In [37]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i]) # 학습이 잘된 pretrained model을 활용하자

## PCA - 2차원

In [38]:
# 차원 확인
pca_test = PCA(random_state=0).fit_transform(vector)

In [39]:
pca_test.shape

(3597, 200)

In [43]:
pca = PCA(n_components=2)
pca_two = pca.fit_transform(vector)

In [44]:
pca_two.shape

(3597, 2)

In [45]:
pca.explained_variance_ratio_[:2]

array([0.07243389, 0.04710615])

In [46]:
# 시각화해보세요!
data = go.Scatter(
    x = pca_two[:, 0],
    y = pca_two[:, 1],
    text = word,
    mode='markers',
    marker = dict(
        size = 3,
        colorscale = 'Jet',
        opacity = 0.8
    )
)

layout = go.Layout(
    autosize = False,
    width = 1000,
    height = 1000,
    margin = go.Margin(
        l = 50,
        r = 50,
        b = 100,
        t = 100,
        pad = 4
    )
)

fig = go.Figure(data = [data], layout = layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




## t-SNE - 2차원

In [50]:
tsne = TSNE(n_components=2, perplexity = 10, learning_rate = 0.05, init = 'random', n_iter = 1000)
tsne_two = tsne.fit_transform(vector)

In [51]:
tsne_two.shape

(3597, 2)

In [52]:
data = go.Scatter(
    x = tsne_two[:, 0],
    y = tsne_two[:, 1],
    text = word,
    mode='markers',
    marker = dict(
        size = 3,
        colorscale = 'Jet',
        opacity = 0.8
    )
)

layout = go.Layout(
    autosize = False,
    width = 1000,
    height = 1000,
    margin = go.Margin(
        l = 50,
        r = 50,
        b = 100,
        t = 100,
        pad = 4
    )
)

fig = go.Figure(data = [data], layout = layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




## t-SNE - 3차원

In [53]:
tsne2 = TSNE(n_components=3, perplexity = 10, learning_rate = 0.05, init = 'random', n_iter = 1000)
tsne_three = tsne2.fit_transform(vector)

In [54]:
data = go.Scatter3d(
    x = tsne_three[:, 0],
    y = tsne_three[:, 1],
    z = tsne_three[:, 2],
    text = word,
    mode='markers',
    marker = dict(
        size = 3,
        colorscale = 'Jet',
        opacity = 0.8
    )
)

layout = go.Layout(
    autosize = False,
    width = 1000,
    height = 1000,
    margin = go.Margin(
        l = 50,
        r = 50,
        b = 100,
        t = 100,
        pad = 4
    )
)

fig = go.Figure(data = [data], layout = layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

=> perplexity가 증가할수록 embedding vector 간에 퍼지는 정도가 감소함을 확인할 수 있다.

## perplexity 5

In [55]:
tsne = TSNE(n_components=2, perplexity = 5, learning_rate = 0.05, init = 'random', n_iter = 1000)
tsne_two = tsne.fit_transform(vector)

In [56]:
data = go.Scatter(
    x = tsne_two[:, 0],
    y = tsne_two[:, 1],
    text = word,
    mode='markers',
    marker = dict(
        size = 3,
        colorscale = 'Jet',
        opacity = 0.8
    )
)

layout = go.Layout(
    autosize = False,
    width = 1000,
    height = 1000,
    margin = go.Margin(
        l = 50,
        r = 50,
        b = 100,
        t = 100,
        pad = 4
    )
)

fig = go.Figure(data = [data], layout = layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




## perplexity 10

In [57]:
tsne = TSNE(n_components=2, perplexity = 10, learning_rate = 0.05, init = 'random', n_iter = 1000)
tsne_two = tsne.fit_transform(vector)

In [58]:
data = go.Scatter(
    x = tsne_two[:, 0],
    y = tsne_two[:, 1],
    text = word,
    mode='markers',
    marker = dict(
        size = 3,
        colorscale = 'Jet',
        opacity = 0.8
    )
)

layout = go.Layout(
    autosize = False,
    width = 1000,
    height = 1000,
    margin = go.Margin(
        l = 50,
        r = 50,
        b = 100,
        t = 100,
        pad = 4
    )
)

fig = go.Figure(data = [data], layout = layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




## perplexity 25

In [61]:
tsne = TSNE(n_components=2, perplexity = 25, learning_rate = 0.05, init = 'random', n_iter = 1000)
tsne_two = tsne.fit_transform(vector)

In [62]:
data = go.Scatter(
    x = tsne_two[:, 0],
    y = tsne_two[:, 1],
    text = word,
    mode='markers',
    marker = dict(
        size = 3,
        colorscale = 'Jet',
        opacity = 0.8
    )
)

layout = go.Layout(
    autosize = False,
    width = 1000,
    height = 1000,
    margin = go.Margin(
        l = 50,
        r = 50,
        b = 100,
        t = 100,
        pad = 4
    )
)

fig = go.Figure(data = [data], layout = layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




## perplexity 50

In [59]:
tsne = TSNE(n_components=2, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000)
tsne_two = tsne.fit_transform(vector)

In [60]:
data = go.Scatter(
    x = tsne_two[:, 0],
    y = tsne_two[:, 1],
    text = word,
    mode='markers',
    marker = dict(
        size = 3,
        colorscale = 'Jet',
        opacity = 0.8
    )
)

layout = go.Layout(
    autosize = False,
    width = 1000,
    height = 1000,
    margin = go.Margin(
        l = 50,
        r = 50,
        b = 100,
        t = 100,
        pad = 4
    )
)

fig = go.Figure(data = [data], layout = layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!